In [10]:
import pandas as pd
import numpy as np

In [11]:
# data
rust_timeseries = {}

for i in range(8):
  rust_timeseries[f"bus_group{i+1}"] = pd.read_excel(
    "rust_timeseries_data.xlsx", sheet_name=i
      )
  
rust_timeseries["bus_group3"].head()

,4338,4339,4340,4341,4342,4343,4344,4345,4346,4347,...,4376,4377,4378,4379,4380,4381,4382,4383,4384,4385
0,3369,16782,16037,12911,8958,3753,4445,17483,3083,19040,...,15879,20423,6130,8550,16019,19368,23614,17306,13301,7064
1,7946,22560,21147,17126,9144,7933,8068,22717,8847,24329,...,15944,24798,9772,12523,20763,24705,29688,21887,17214,10271
2,11601,25244,24279,18101,12603,11187,10969,25193,11934,26793,...,17052,28037,13120,15695,21317,28105,32642,25400,20275,10842
3,17056,28866,28275,18101,16195,15094,14785,29685,16177,31133,...,19673,30670,15771,19114,21317,31219,36837,28516,23615,11708
4,21485,32449,31725,18101,19048,18678,18223,32767,19670,33949,...,22726,34274,19336,22883,23853,34310,40351,32224,27047,11789


## 遷移行列の推定

*  x1,x2導出
    1. ビンを作成(1,2, ..., 21)．
        - ビン20までのデータで推定可能なので，ビン21を，欠損値に変換．
    2. 下から上の差分取る．
    3. 0, 1の数を数え，リストに収納．
    4. この操作を全てのバスグループで行い，リストを合計する．
    5. x1,x2がわかる．

* x3の導出
    1. ビンを作成(1,2, ..., 21)．
    2. 差分が2のデータを収集．
    3. 足してx3がわかる．

* x4の導出
    1. 20→21のデータを数える． 
    2. x4がわかる．

In [12]:
# x1,x2
total_counts = [0, 0]

for i in range(8):
  df = rust_timeseries[f"bus_group{i+1}"]
  for j in range(20):
    lower_bound = j * 5000
    upper_bound = (j + 1) * 5000
    df = df.applymap(lambda x: j+1 if lower_bound <= x < upper_bound else x)
    
  # 21個目のビンを欠損値に変換
  df = df.applymap(lambda x: np.nan if 20*5000 <= x else x)
  
  df = df.diff()

  counts = [(df == val).sum().sum() for val in [0, 1]]
  total_counts = [total_counts[i] + counts[i] for i in range(len(total_counts))]

x1,x2 = total_counts
x1,x2

(982, 2175)

In [13]:
# x3
total_counts = [0]

for i in range(8):
  df = rust_timeseries[f"bus_group{i+1}"]
  for j in range(20):
    lower_bound = j * 5000
    upper_bound = (j + 1) * 5000
    df = df.applymap(lambda x: j+1 if lower_bound <= x < upper_bound else x)
    df = df.applymap(lambda x: 21 if 20*5000 <= x else x) # ビン21の処理
    
  df = df.diff()

  counts = [(df == val).sum().sum() for val in [2]]
  total_counts = [total_counts[i] + counts[i] for i in range(len(total_counts))]

x3 = total_counts[0]
x3

61

In [14]:
# x4
total_counts = [0]

for i in range(8):
  df = rust_timeseries[f"bus_group{i+1}"]

  # 20, 21個目のビンだけ処理
  df = df.applymap(lambda x: 20 if 19*5000 <= x < 20*5000 else x)  
  df = df.applymap(lambda x: 21 if 20*5000 <= x  else x)
  df = df.applymap(lambda x: x if x == 20 or x ==21 else np.nan)

  df = df.diff()

  counts = [(df == val).sum().sum() for val in [1]]
  total_counts = [total_counts[i] + counts[i] for i in range(len(total_counts))]

x4 = total_counts[0]
x4

119

In [15]:
print(x1,x2,x3,x4)

982 2175 61 119


In [16]:
kappa1 = (x2*(x2 + x3 + x4))/((x2 + x3)*(x1 + x2 + x3 + x4))
kappa2 = (x3*(x2 + x3 + x4))/((x2 + x3)*(x1 + x2 + x3 + x4))
print(kappa1,kappa2)

0.6864709552944355 0.019252748631246236
